In [1]:
import pickle

def load_models():

    loaded_station_names = ['LIVST', 'STFD', 'SHENFLD', 'CHLMSFD', 'CLCHSTR', 'MANNGTR', 'IPSWICH', 'STWMRKT', 'DISS']
    # Load the models from files
    loaded_model_dict = {}
    for station_name in loaded_station_names:
        with open(f'{station_name}_model_knn.pkl', 'rb') as model_file:
            loaded_model_dict[station_name] = pickle.load(model_file)
    
    return loaded_model_dict
#MANUALLY CHANGE THE OPENING NAME
loaded_models = load_models()
print(loaded_models)

{'LIVST': KNeighborsRegressor(algorithm='brute', leaf_size=10, metric='euclidean',
                    n_neighbors=31, p=1, weights='distance'), 'STFD': KNeighborsRegressor(algorithm='ball_tree', leaf_size=20, metric='euclidean',
                    n_neighbors=25, p=1, weights='distance'), 'SHENFLD': KNeighborsRegressor(algorithm='ball_tree', leaf_size=20, metric='manhattan',
                    n_neighbors=51, p=1, weights='distance'), 'CHLMSFD': KNeighborsRegressor(algorithm='ball_tree', leaf_size=40, metric='manhattan',
                    n_neighbors=51, p=1), 'CLCHSTR': KNeighborsRegressor(algorithm='brute', leaf_size=10, metric='euclidean',
                    n_neighbors=51, p=1), 'MANNGTR': KNeighborsRegressor(algorithm='ball_tree', leaf_size=10, metric='manhattan',
                    n_neighbors=51, p=1), 'IPSWICH': KNeighborsRegressor(algorithm='brute', leaf_size=10, metric='euclidean',
                    n_neighbors=41, p=1), 'STWMRKT': KNeighborsRegressor(algorithm='brut

In [12]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def parameterise(lateness, hour, day_of_week):
    "Encodes the input params into machine-readable format"
    def group_hours(hour_func):
        for i, (start, end) in enumerate(ranges):
            if start <= hour_func <= end:
                return f'Group_{i + 1}'
        return 'Other'
    is_weekend = 0
    is_offpeak = 0
    
    if (day_of_week == 'SAT') or(day_of_week == 'SUN'):
        is_weekend = 1
        is_offpeak = 1
    if (hour >= 9) or (hour <= 2):
        is_offpeak = 1
    ranges = [(6,9),(10, 15), (16, 19), (20, 23),(0,5)]
    group = group_hours(hour)
    
    one_hot_array = [0] * (len(ranges) )  # Initialize array with zeros
    if group != 'Other':
        group_index = int(group.split('_')[1]) - 1
        one_hot_array[group_index] = 1  # Set the corresponding index to 1
    else:
        one_hot_array[-1] = 1  # Set the last index to 1 for 'Other' group
        
    days_of_week = ['FRI','MON','SAT','SUN','THURS','TUES','WEDS']
    one_hot_day = [0] * len(days_of_week)  # Initialize array with zeros
    if day_of_week in days_of_week:
        index = days_of_week.index(day_of_week)
        one_hot_day[index] = 1  # Set the corresponding index to 1
    output = [lateness, is_weekend, is_offpeak]
    for val in one_hot_array:
        output.append(val)
    for val in one_hot_day:
        output.append(val)
    output = np.array(output)    
    output = output.reshape(1,-1)
    return output
    
    
print(parameterise(-10,10,'SAT'))  

def predict_lateness(current_station, target,lateness, hour, day, models):
    stations = ['LIVST', 'STFD', 'SHENFLD', 'CHLMSFD', 'CLCHSTR', 'MANNGTR', 'IPSWICH', 'STWMRKT', 'DISS', 'NRCH']
    starting_index = stations.index(current_station)
    #starting_index = starting_index + 1
    #parameters = np.array([600,1,1,2,2]) # lateness-weekend-offpeak-day-hour 
    parameters = parameterise(lateness, hour, day)

    next_station = stations[starting_index]
    target = 'NRCH'

    while next_station != target:  # Stop when you reach target
        # Predict lateness at the next station
        next_lateness = models[next_station].predict(parameters)
        parameters[0][0] = next_lateness

        starting_index = starting_index + 1
        next_station = stations[starting_index]

        # Print the predicted lateness for the next station
        print(f"Predicted lateness at {next_station}: ",parameters[0][0], "minutes")
        
        
        
target = 'NRCH'
current_station = 'LIVST' 
lateness = 60
hour = 18
day='SAT'
predict_lateness(current_station, target, lateness,hour,day, loaded_models)

[[-10   1   1   0   1   0   0   0   0   0   1   0   0   0   0]]
Predicted lateness at STFD:  57 minutes
Predicted lateness at SHENFLD:  56 minutes
Predicted lateness at CHLMSFD:  32 minutes
Predicted lateness at CLCHSTR:  19 minutes
Predicted lateness at MANNGTR:  20 minutes
Predicted lateness at IPSWICH:  19 minutes
Predicted lateness at STWMRKT:  19 minutes
Predicted lateness at DISS:  21 minutes
Predicted lateness at NRCH:  19 minutes
